# Google Colab setup with Google Drive folder

This notebook provides the code you need to set up Google Colab to run and import files from within a Google Drive folder.

This will allow you to upload assignment code to your Google Drive and then run the code on Google Colab machines (with free GPUs if needed). 

You will need to create a folder in your Google Drive to hold your assignments and you will need to open Colaboratory within this folder before running the set up code (check the link above to see how).

# Mount Google Drive

This will allow the Colab machine to access Google Drive folders by mounting the drive on the machine. You will be asked to copy and paste an authentication code.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
ls

gdrive/  sample_data/


# Change directory to allow imports


As noted above, you should create a Google Drive folder to hold all your assignment files. You will need to add this code to the top of any python notebook you run to be able to import python files from your drive assignment folder (you should change the file path below to be your own assignment folder).

In [0]:
import os
os.chdir("/content/gdrive/My Drive/CS598")

In [25]:
ls # Check if this is your MP4 folder

RADVESS/


# Copy data to local dir

In [0]:
!mkdir /dataCS5981
!cp -r RADVESS/ /data
# !tar -xf /data/cifar100.tar.gz -C /data/
# !cp data/test.tar.gz /data
# !tar -xf /data/test.tar.gz -C /data
# !cp data/train.tar.gz /data
# !tar -xf /data/train.tar.gz -C /data/

In [34]:
ls /data/RADVESS

Actor_01/  Actor_06/  Actor_11/  Actor_16/  Actor_21/
Actor_02/  Actor_07/  Actor_12/  Actor_17/  Actor_22/
Actor_03/  Actor_08/  Actor_13/  Actor_18/  Actor_23/
Actor_04/  Actor_09/  Actor_14/  Actor_19/  Actor_24/
Actor_05/  Actor_10/  Actor_15/  Actor_20/  Audio_Speech_Actors_01-24.zip


# Set up GPU and PyTorch

First, ensure that your notebook on Colaboratory is set up to use GPU. After opening the notebook on Colaboratory, go to Edit>Notebook settings, select Python 3 under "Runtime type," select GPU under "Hardware accelerator," and save.

Next, install PyTorch:

In [36]:
!pip3 install torch torchvision
!pip3 install librosa

Make sure that pytorch is installed and works with GPU:

In [6]:
import torch
a = torch.Tensor([1]).cuda()
print(a)


tensor([1.], device='cuda:0')


In [0]:
import numpy as np
import scipy.linalg as lg
from scipy import signal
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import scipy
import sys
import os
from random import shuffle
import librosa
import librosa.display

In [7]:
torch.cuda.is_available()

True

In [8]:
test(model, criterion)

NameError: ignored

In [0]:
data_names = []
data_labels = []
data_mfccs = []

In [0]:
emotionDict = {'happy': 0, 'sad': 1, 'angry': 2, 'fearful': 3}

for dirs in os.listdir('/data/RADVESS/'):
    if os.path.isdir('/data/RADVESS/'+ dirs):
      files = os.listdir('/data/RADVESS/'+ dirs)
    else:
      continue
    for file in files:
        f = file.split('-')
        inEmotion = False
        if(f[2]=='03'):
            data_labels.append(emotionDict['happy'])
            inEmotion = True
        elif(f[2]=='04'):
            data_labels.append(emotionDict['sad'])
            inEmotion = True
        elif(f[2]=='05'):
            data_labels.append(emotionDict['angry'])
            inEmotion = True
        elif(f[2]=='06'):
            data_labels.append(emotionDict['fearful'])
            inEmotion = True
        if inEmotion == True:
            data_names.append(file)
            # load data
            data, sample_rate = librosa.load('RADVESS/'+dirs+'/'+ file, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5) # data is 110250
            # get mfcc features
            mfccs = librosa.feature.mfcc(y=data.astype(dtype=float), sr=sample_rate, n_mfcc=13) # 13 x 216
            data_mfccs.append(mfccs)

In [49]:
# Convert 2d mfcc features to 1d
features = [np.mean(mfcc, axis=0) for mfcc in data_mfccs] # (216,)
# features = features.astype(dtype='double')
print(len(features), features[0].shape, len(data_names), len(data_labels))

768 (216,) 768 768


In [50]:
# shuffle data
z = list(zip(features, data_labels))
shuffle(z)
features, data_labels = zip(*z)

# split data into train and test set
train_feature = features[0:int(len(features)*0.8)]
test_feature = features[int(len(features)*0.8):]
train_label = data_labels[0:int(len(data_labels)*0.8)]
test_label = data_labels[int(len(data_labels)*0.8):]
print(len(train_feature),len(test_feature),len(train_label),len(test_label))

# Convert list to array
train_feature = np.array(train_feature)
test_feature = np.array(test_feature)
print(train_feature.shape, train_feature[0].shape)
print(len(train_label))

614 154 614 154
(614, 216) (216,)
614


In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.utils.data
# import torchvision
# import torchvision.transforms as transforms

from torch.autograd import Variable
IS_GPU = True


In [0]:
class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        # Add more conv layers with increasing output channels
        # Add normalization layers after conv layers (nn.BatchNorm2d)
        # Also experiment with kernel size in conv2d layers (say 3
        # inspired from VGGNet)
    
        self.conv_net = nn.Sequential(
            nn.Conv1d(1, 256, kernel_size=5, padding=2), # (216 + 2*2 - 1*(5-1) - 1)/1+1= 216
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            
            nn.Conv1d(256, 128, kernel_size=5, padding=2), # (216 + 2*2 - 1*(5-1) - 1)/1+1= 216
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            
            nn.Dropout(p=0.1),
            
            nn.MaxPool1d(kernel_size=8), # 216/8=27
            
            nn.Conv1d(128, 128, kernel_size=5, padding=2), # (27 + 2*2 - 1*(5-1) - 1)/1+1= 27
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            
            nn.Conv1d(128, 128, kernel_size=5, padding=2), # (27 + 2*2 - 1*(5-1) - 1)/1+1= 27
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            
            nn.Flatten(),
        )

        # Add more linear (fc) layers
        # Add normalization layers after linear and
        # experiment inserting them before or after ReLU (nn.BatchNorm1d)
        
        self.fc_net = nn.Sequential(
            nn.Linear(27*128,4),
            nn.Softmax(),
        )

    def forward(self, x):

        x = self.conv_net(x)
        x = self.fc_net(x)
        return x

# Create an instance of the nn.module class defined above:
net = BaseNet()
net = net.float()
if IS_GPU:
    net = net.cuda()

In [0]:
########################################################################
# 3. Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Here we use Cross-Entropy loss and SGD with momentum.
# The CrossEntropyLoss criterion already includes softmax within its
# implementation. That's why we don't use a softmax in our model
# definition.

import torch.optim as optim
criterion = nn.CrossEntropyLoss()

# Tune the learning rate.
# See whether the momentum is useful or not
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

plt.ioff()
fig = plt.figure()
train_loss_over_epochs = []
val_accuracy_over_epochs = []

In [63]:
# Train the network
n_batches = 32
EPOCHS = 20

for epoch in range(EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(int(train_feature.shape[0]/n_batches)+1):
        # Local batches and labels
        if (i+1)*n_batches > train_feature.shape[0]:
            inputs, labels = train_feature[i*n_batches:,:], train_label[i*n_batches:]
        else:
            inputs, labels = train_feature[i*n_batches:(i+1)*n_batches,:], train_label[i*n_batches:(i+1)*n_batches]

        inputs = torch.from_numpy(inputs)
        labels = torch.from_numpy(np.array(labels))

        if IS_GPU:
            inputs = inputs.cuda()
            labels = labels.cuda()

        # wrap them in Variable
        inputs = Variable((inputs))
        labels = Variable(((labels)))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        inputs = torch.unsqueeze(inputs,1)
        outputs = net(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    # Normalizing the loss by the total number of train batches
    running_loss/=int(train_feature.shape[0]/n_batches)+1
    print('[%d] loss: %.3f' %
          (epoch + 1, running_loss))

    # Scale of 0.0 to 100.0
    # Calculate validation set accuracy of the existing model
#     val_accuracy, val_classwise_accuracy = \
#         calculate_val_accuracy(valloader, IS_GPU)
#     print('Accuracy of the network on the val images: %d %%' % (val_accuracy))

    train_loss_over_epochs.append(running_loss)
#     val_accuracy_over_epochs.append(val_accuracy)
# -----------------------------


# Plot train loss over epochs and val set accuracy over epochs
# Nothing to change here
# -------------
plt.subplot(2, 1, 1)
plt.ylabel('Train loss')
plt.plot(np.arange(EPOCHS), train_loss_over_epochs, 'k-')
plt.title('train loss and val accuracy')
plt.xticks(np.arange(EPOCHS, dtype=int))
plt.grid(True)

# plt.subplot(2, 1, 2)
# plt.plot(np.arange(EPOCHS), val_accuracy_over_epochs, 'b-')
# plt.ylabel('Val accuracy')
# plt.xlabel('Epochs')
# plt.xticks(np.arange(EPOCHS, dtype=int))
# plt.grid(True)
# plt.savefig("plot.png")
# plt.close(fig)
print('Finished Training')
# -------------

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[1] loss: 1.310
[2] loss: 1.225
[3] loss: 1.197
[4] loss: 1.139
[5] loss: 1.093
[6] loss: 1.072
[7] loss: 1.034
[8] loss: 1.028
[9] loss: 0.994
[10] loss: 0.999
[11] loss: 0.972
[12] loss: 0.955
[13] loss: 0.950
[14] loss: 0.935
[15] loss: 0.915
[16] loss: 0.901
[17] loss: 0.898
[18] loss: 0.890
[19] loss: 0.885
[20] loss: 0.906
Finished Training


In [65]:
########################################################################
# 5. Try the network on test data, and create .csv file
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
########################################################################

# Check out why .eval() is important!
# https://discuss.pytorch.org/t/model-train-and-model-eval-vs-model-and-model-eval/5744/2
net.eval()

total = 0
predictions = []
# for i in range(test_feature.shape[0]):
data, label = test_feature, test_label

data = torch.from_numpy(data)
data = torch.unsqueeze(data,1)
print(data.shape)

if IS_GPU:
  data = data.cuda()
  # labels = labels.cuda()

data = Variable(data)
outputs = net(data.float())
_, predicted = torch.max(outputs.data, 1)
predictions.extend(list(predicted.cpu().numpy()))
#     total += label.size(0)
acc = 0
for l_i, label in enumerate(predictions):
    if label == test_label[l_i]:
        acc += 1
#     print([str(l_i), str(label)], test_label[l_i])
print("accuracy", acc/len(test_label))

torch.Size([154, 1, 216])
accuracy 0.5714285714285714


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
